In [1]:
import sys
sys.path.append("/root/HSE_diploma/")
sys.path.append("/root/HSE_diploma/evaluator/")

In [2]:
sys.path

['/root/HSE_diploma/ipynb',
 '/opt/conda/lib/python310.zip',
 '/opt/conda/lib/python3.10',
 '/opt/conda/lib/python3.10/lib-dynload',
 '',
 '/opt/conda/lib/python3.10/site-packages',
 '/opt/conda/lib/python3.10/site-packages/mpmath-1.2.1-py3.10.egg',
 '/root/HSE_diploma/',
 '/root/HSE_diploma/evaluator/']

In [3]:
import os
import json

import transformers
from peft import PeftModel
from transformers import LlamaForCausalLM as LLaMAForCausalLM
from transformers import LlamaTokenizer as LLaMATokenizer
from peft import prepare_model_for_int8_training, PrefixTuningConfig, get_peft_model

from datasets import load_dataset
from EvaluateTestSet import EvaluateTestSet
from transformers.integrations import TensorBoardCallback
from transformers import GenerationConfig



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)


/root/HSE_diploma/prompter/templates/


In [4]:
# peft_config = PrefixTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=30)


# # 训练器配置
# p_type = "lora"
# if p_type == "prefix-tuning":
#   peft_type = PeftType.PREFIX_TUNING
#   peft_config = PrefixTuningConfig(task_type="SEQ_CLS", num_virtual_tokens=20)
# elif p_type == "prompt-tuning":
#   peft_type = PeftType.PROMPT_TUNING
#   peft_config = PromptTuningConfig(task_type="SEQ_CLS", num_virtual_tokens=20)
# elif p_type == "p-tuning":
#   peft_type = PeftType.P_TUNING
#   peft_config = PromptEncoderConfig(task_type="SEQ_CLS", num_virtual_tokens=20, encoder_hidden_size=128)
# elif p_type == "lora":
#   peft_type = PeftType.LORA
#   peft_config = LoraConfig(task_type="SEQ_CLS", inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1)
# # print(peft_type)

In [7]:

def init_prefix_model_and_tokenizer(default_model,
                                    task_type = "CAUSAL_LM",
                                    num_virtual_tokens = 30
                                  ):


    """
        
    """
    model = LLaMAForCausalLM.from_pretrained(
    default_model,
    load_in_8bit=True,
    device_map="auto",
    )
    tokenizer = LLaMATokenizer.from_pretrained(
        default_model, add_eos_token=True
    )

    model = prepare_model_for_int8_training(model)

    config = PrefixTuningConfig(task_type = task_type,
                                num_virtual_tokens = num_virtual_tokens,
                                prefix_projection = True, 
                                encoder_hidden_size = 1024,
                               )
    

#     PrefixTuningConfig(
#         r=LORA_R,
#         lora_alpha=LORA_ALPHA,
#         target_modules=["q_proj", "v_proj"],
#         lora_dropout=LORA_DROPOUT,
#         bias="none",
#         task_type="CAUSAL_LM",
#     )

    model = get_peft_model(model, config)

    tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token

    return model, tokenizer



# class MyCustomCallback(TensorBoardCallback):
#     #log_bleu_steps_factor = 5
#     bleu_generation_max_new_tokens = 30
#     bleu_fn_test_data = "/root/data/t2c_answers.json"
#     bleu_fn_etalon = "/root/data/answers.json"
#     log_step = 0
    
#     def on_log(self, args, state, control, logs=None, **kwargs):
#         super().on_log(args, state, control, logs=logs, **kwargs)
#         #print("kwargs", len(kwargs), kwargs.keys())
#         if self.tb_writer is not None:
#             #print(state)
#             #print(state.global_step)
#             #print(self.log_step)
#             if (self.log_step % self.log_bleu_steps_factor ==0):
#                 model = kwargs['model']
#                 tokenizer = kwargs['tokenizer']
                
#                 model.eval()
#                 assert not model.training
#                 generation_config = GenerationConfig(max_new_tokens = self.bleu_generation_max_new_tokens,
#                                                      # min_new_tokens = 5,
#                                                      temperature = 1.0
#                                                     )
#                 print("generation_config:", generation_config)
#                 evaluator = EvaluateTestSet(generation_config = generation_config,
#                                             fn_test_data = self.bleu_fn_test_data,
#                                             fn_etalon = self.bleu_fn_etalon,
#                                             batch_size = 1
#                                        )

#                 metric_res = evaluator.evaluate(model=model, 
#                                                 tokenizer=tokenizer,
#                                                )
#                 model.train()
#                 assert model.training
#                 print(metric_res)
#                 for key, val in metric_res.items():
#                     #add "custom/something"
#                     self.tb_writer.add_scalar(key, val, state.global_step)
#                 self.tb_writer.flush()
#             self.log_step += 1

In [8]:
CONFIG_PATH = "/root/HSE_diploma/experiments_configs/"
EXPERIMENTS_PATH = "/root/experiments/"
experiment_name = "t2c_concode_220428_v35_prefix"
# t2c_concode_220428_v18.json

In [9]:
current_config_path = os.path.join(CONFIG_PATH, experiment_name + "_config.json")
experiment_config = json.load(open(current_config_path, "r"))

assert experiment_config['experiment_name'] == experiment_name

In [10]:
experiment_config['resume_from_checkpoint'] = False

In [11]:
assert("LORA_R" not in experiment_config)
assert("LORA_ALPHA" not in experiment_config)
assert("LORA_DROPOUT" not in experiment_config)

In [12]:
assert experiment_config['resume_from_checkpoint'] == False

In [13]:
assert experiment_config['experiment_name'] == experiment_name

In [14]:
current_experiment_path = os.path.join(EXPERIMENTS_PATH, experiment_name)

In [15]:
print("current_experiment_path", current_experiment_path)

current_experiment_path /root/experiments/t2c_concode_220428_v35_prefix


In [16]:
!mkdir {current_experiment_path}

mkdir: cannot create directory ‘/root/experiments/t2c_concode_220428_v35_prefix’: File exists


In [17]:
json.dump(experiment_config, open(current_experiment_path + \
                                  "/experiment_config.json", 
                                  "w+"
                                 )
         )

In [18]:
# setattr(MyCustomCallback, "log_bleu_steps_factor", experiment_config['log_bleu_steps_factor'])
# MyCustomCallback.log_bleu_steps_factor

In [19]:
model, tokenizer = init_prefix_model_and_tokenizer(default_model = experiment_config["default_model"],
                                                 task_type = experiment_config['PREFIX_task_type'],
                                                 num_virtual_tokens = experiment_config['PREFIX_num_virtual_tokens']
                                                )


data = load_dataset("json", 
                    data_files = {"train": experiment_config["fn_train_dataset"],
                                  "eval":  experiment_config["fn_eval_dataset"]
                                 }
                   )

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Found cached dataset json (/root/.cache/huggingface/datasets/json/default-02cbc528b265cbf4/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
def verbose_model_tokenizer(model, tokenizer):
    print("tokenizer.pad_token_id", tokenizer.pad_token_id)
    print("tokenizer.eos_token_id", tokenizer.eos_token_id)
    print("tokenizer.bos_token_id", tokenizer.bos_token_id)
    print("tokenizer.unk_token_id", tokenizer.unk_token_id)

    print("model.config.pad_token_id", model.config.pad_token_id)
    print("model.config.eos_token_id", model.config.eos_token_id)
    print("model.config.bos_token_id", model.config.bos_token_id)
    model.print_trainable_parameters()
    
verbose_model_tokenizer(model, tokenizer)

tokenizer.pad_token_id 0
tokenizer.eos_token_id 2
tokenizer.bos_token_id 1
tokenizer.unk_token_id 0
model.config.pad_token_id 0
model.config.eos_token_id 2
model.config.bos_token_id 1
trainable params: 273015808 || all params: 7011431424 || trainable%: 3.893866908053496


In [21]:
experiment_config

{'experiment_name': 't2c_concode_220428_v35_prefix',
 'fn_train_dataset': '/root/data/t2c_train.json',
 'fn_eval_dataset': '/root/data/t2c_answers.json',
 'default_model': 'yahma/llama-7b-hf',
 'MICRO_BATCH_SIZE': 2,
 'BATCH_SIZE': 10,
 'EPOCHS': 2,
 'LEARNING_RATE': 0.0002,
 'CUTOFF_LEN': 256,
 'PREFIX_task_type': 'CAUSAL_LM',
 'PREFIX_num_virtual_tokens': 30,
 'warmup_steps': 200,
 'fp16': True,
 'logging_steps': 10,
 'eval_steps': 100,
 'evaluation_strategy': 'steps',
 'save_total_limit': 1,
 'save_strategy': 'steps',
 'save_steps': 500,
 'seed': 42,
 'logging_strategy': 'steps',
 'report_to': 'tensorboard',
 'mlm': False,
 'truncation': True,
 'padding': 'max_length',
 'config_use_cache': False,
 'resume_from_checkpoint': False,
 'bleu_batch_size': 5,
 'GRADIENT_ACCUMULATION_STEPS': 5,
 'log_bleu_steps_factor': 50,
 'load_best_model_at_end': False}

In [22]:
from prompter import generate_prompt
# from prompter import Prompter
# prompter = Prompter()

# def generate_prompt(data_point):
#     if "input" in data_point and data_point["input"]:
#         return prompter.generate_prompt(instruction = data_point["instruction"],
#                                         input = data_point["input"],
#                                         label = data_point["output"]
#                                        )
#     else:
#         return prompter.generate_prompt(instruction = data_point["instruction"],
#                                         #input = None,
#                                         label = data_point["output"]
#                                        )

In [23]:
experiment_config["truncation"],\
experiment_config["CUTOFF_LEN"],\
experiment_config["padding"]

(True, 256, 'max_length')

In [22]:
model.to("cuda")

PeftModelForCausalLM(
  (base_model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32000, 4096, padding_idx=0)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaAttention(
            (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (v_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
            (down_proj): Linear8bitLt(in_features=11008, out_features=4096, bias=False)
            (up_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
            (act_fn): SiLUActivation()
          )
          (inpu

In [23]:
assert "return_attention_mask" not in experiment_config

In [25]:
data = data.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=experiment_config["truncation"],
        max_length=experiment_config["CUTOFF_LEN"],
        padding=experiment_config["padding"],
#         return_attention_mask = experiment_config['return_attention_mask']
    )
)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [25]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [30]:
f = transformers.DataCollatorForLanguageModeling(tokenizer, 
                                                               mlm=experiment_config["mlm"]
                                                              )

In [26]:
trainer = transformers.Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=data["train"],#train
    eval_dataset=data['eval'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=experiment_config["MICRO_BATCH_SIZE"],
        gradient_accumulation_steps=experiment_config["GRADIENT_ACCUMULATION_STEPS"],
        warmup_steps=experiment_config["warmup_steps"],
        num_train_epochs=experiment_config["EPOCHS"],
        learning_rate=experiment_config["LEARNING_RATE"],
        fp16=experiment_config["fp16"],
        logging_steps=experiment_config["logging_steps"],        
        evaluation_strategy = experiment_config['evaluation_strategy'],
        eval_steps=experiment_config["eval_steps"],
        output_dir=current_experiment_path,#"lora-alpaca",
        save_total_limit=experiment_config["save_total_limit"],
        save_strategy = experiment_config["save_strategy"],
        
        save_steps = experiment_config["save_steps"],
        seed=experiment_config["seed"],
        logging_dir=current_experiment_path,
        logging_strategy=experiment_config["logging_strategy"],
        report_to=experiment_config["report_to"],
        load_best_model_at_end = experiment_config["load_best_model_at_end"]
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, 
                                                               mlm=experiment_config["mlm"]
                                                              ),
#     callbacks = [MyCustomCallback]
)
model.config.use_cache = experiment_config["config_use_cache"]
# print(len(trainer.optimizer.state['found_inf_per_device']))


trainer.train(resume_from_checkpoint=experiment_config["resume_from_checkpoint"])

model.save_pretrained(current_experiment_path)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [40]:
!nvidia-smi

Fri May 19 21:05:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:41:00.0 Off |                  N/A |
| 30%   41C    P2    98W / 350W |   9369MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [31]:
model.device

device(type='cuda', index=0)

In [22]:
!df -h .

Filesystem      Size  Used Avail Use% Mounted on
overlay         118G   22G   97G  19% /


In [23]:
!ls {current_experiment_path}

1684518421.931266   events.out.tfevents.1684518421.c71a00d70d2c.1489.0
1684518421.9377556  events.out.tfevents.1684518421.c71a00d70d2c.1489.2
1684518911.7628372  events.out.tfevents.1684518911.c71a00d70d2c.1622.0
1684518911.768738   events.out.tfevents.1684518911.c71a00d70d2c.1622.2
checkpoint-500	    experiment_config.json


In [24]:
model.save_pretrained(current_experiment_path)

In [3]:
!ls /root/experiments/t2c_concode_220428_v34

1684518421.931266    checkpoint-500
1684518421.9377556   events.out.tfevents.1684518421.c71a00d70d2c.1489.0
1684518911.7628372   events.out.tfevents.1684518421.c71a00d70d2c.1489.2
1684518911.768738    events.out.tfevents.1684518911.c71a00d70d2c.1622.0
adapter_config.json  events.out.tfevents.1684518911.c71a00d70d2c.1622.2
adapter_model.bin    experiment_config.json
